In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import json 
from dotenv import load_dotenv
load_dotenv()
import requests
from urllib import parse


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from code_utils.utils import type_score,get_year_ipcc,get_year_ipbes,get_xy,check_doi_glutton,normalize,aplatir,remove_too_long
from code_utils.BSO import update_bso_publications,get_bso_publications
from code_utils.pickle import load_cache

In [2]:
df_ipcc=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipcc_visualization.json')
ipcc_chap=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\ipcc_chap.jsonl', lines=True)

In [3]:
df_ipcc

,year,url,doi,title,author,freq,countries,concepts,sdg,topics,...,title_OA,year_OA,authors_name,rors,institutions_names,locations_names,locations_ids,type,type_crossref,test_glutton
0,2019,https://www.frontiersin.org/research-topics/82...,10.3389/978-2-88963-118-6,{OceanObs'19: An Ocean of Opportunity. Volume 1},None,1,[None],"[Volume (thermodynamics), Oceanography, Enviro...","[{'id': '14', 'name': 'Life below water'}]",[Marine Biology and Ecology Research],...,Oceanobs'19: An Ocean of Opportunity. Volume I,2020,[None],[None],[None],Frontiers research topics,1664-8714,book,book,True
1,2000,None,10.1007/978-94-015-9612-1,"{Eduard Br{\""{u}}ckner – The Sources and Conse...",None,1,"[CA, UA]","[Geographer, Climate change, Politics, Climato...","[{'id': '13', 'name': 'Climate action'}]","[Geology and Paleoclimatology Research, Climat...",...,Eduard Brückner — The Sources and Consequences...,2000,"[[Nico Stehr, [CA]], [Hans von Storch, [UA]]]","[[https://ror.org/03rmrcq20, CA], [https://ror...","[[University of British Columbia, CA], [Instit...",None,None,book,book,True
2,1900,None,10.1002/andp.19003081208,"{{\""{U}}ber die Bedeutung des Wasserdampfes un...","{\AA}ngstr{\""{o}}m, Knut",1,[US],"[Physics, Humanities, Angstrom, Crystallograph...",None,"[Atmospheric Ozone and Climate, Atmospheric ch...",...,Ueber die Bedeutung des Wasserdampfes und der ...,1900,"[[Knut Ångström, [US]]]","[[https://ror.org/03wbhj477, US]]","[[Upsala College, US]]",Annalen der Physik,0003-3804,article,journal-article,True
3,1964,https://www.tandfonline.com/doi/full/10.3402/t...,10.3402/tellusa.v16i1.8885,{The parameters of atmospheric turbidity},"{\AA}ngstr{\""{o}}m, Anders",1,[US],"[Turbidity, Environmental science, Meteorology...",None,[Ocean Waves and Remote Sensing],...,The parameters of atmospheric turbidity,1964,"[[Anders Ångström, [US]]]","[[https://ror.org/03a8x4b06, US]]","[[Eppley Foundation for Research, US]]",Tellus A Dynamic Meteorology and Oceanography,0280-6495,article,journal-article,True
4,1929,https://www.tandfonline.com/doi/full/10.1080/2...,10.1080/20014422.1929.11880498,{On the Atmospheric Transmission of Sun Radiat...,"{\AA}ngstr{\""{o}}m, Anders",1,[None],"[Atmospheric dust, Radiation, Transmission (te...","[{'id': '14', 'name': 'Life below water'}]",[Atmospheric aerosols and clouds],...,On the Atmospheric Transmission of Sun Radiati...,1929,"[[Anders Ångström, []]]",[],[],Geografiska Annaler,1651-3215,article,journal-article,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72829,2018,https://doi.org/10.1016%2Fj.apenergy.2017.07.036,10.1016/j.apenergy.2017.07.036,The water-food-energy nexus optimization appro...,Jie Zhang and Pietro Elia Campana and Tian Yao...,1,"[SE, US]","[Nexus (standard), Agriculture, Yield (enginee...",None,"[Water-Energy-Food Nexus Studies, Water resour...",...,The water-food-energy nexus optimization appro...,2017,"[[Jie Zhang, [US]], [Pietro Elia Campana, [SE]...","[[https://ror.org/047s2c258, US], [https://ror...","[[University of Maryland, College Park, US], [...",Applied Energy,0306-2619,article,journal-article,True
72830,2019,https://doi.org/10.1029%2F2019rg000644,10.1029/2019rg000644,A Review of the Role of the Atlantic Meridiona...,Rong Zhang and Rowan Sutton and Gokhan Danabas...,1,"[GB, US]","[Atlantic multidecadal oscillation, Climatolog...","[{'id': '13', 'name': 'Climate action'}]","[Climate variability and models, Tropical and ...",...,A Review of the Role of the Atlantic Meridiona...,2019,"[[Rong Zhang, [US]], [Rowan Sutton, [GB]], [Gö...","[[https://ror.org/03vmn1898, US], [https://ror...","[[NOAA Geophysical Fluid Dynamics Laboratory, ...",Reviews of Geophysics,1944-9208,review,journal-article,True
72831,2013,https://doi.org/10.1016%2Fj.enpol.2013.08.092,10.1016/j.enpol.2013.08.092,To what extent does wind power deployment affe...,Xiaoli Zhao and Sufang Zhang and Yasheng Zou a...,1,[CN],"[Software deployment, Revenue, Renewable energ...","[{'id': 

In [4]:
ipcc_chap['ipcc']=ipcc_chap.apply(lambda row: {'name': str(row.chap), 'wg': str(row.wg), 'chap': str(row.chap)[-2:]},axis=1)

In [5]:
ipcc_chap=ipcc_chap.groupby(by='doi').agg({'ipcc': lambda x: list(x)}).reset_index()

In [6]:
df_ipcc.loc[:,'rors']=df_ipcc.loc[:,'rors'].apply(lambda x: [[]] if x==[None] else x)
df_ipcc.loc[:,'rors']=df_ipcc.loc[:,'rors'].apply(lambda x: None if aplatir(x)==[] else x)

In [7]:
df_ipcc_group=df_ipcc.groupby(by=['doi'], dropna=True).agg({'year': lambda x: list(x)[0], 'title': lambda x: list(x)[0], 'rors': lambda x: list(x)[0],'authors_name': lambda x: list(x)[0], 'institutions_names': lambda x: list(x)[0], 'countries': lambda x: list(x)[0], 'topics': lambda x: list(x)[0], 'freq':'sum'})
df_ipcc_group=df_ipcc_group.reset_index()

In [8]:
df_ipcc_group=pd.merge(df_ipcc_group,ipcc_chap, on='doi', how='left')

In [9]:
df_ipcc_group.loc[:,'countries']=df_ipcc_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [10]:
len(df_ipcc_group)

52471

In [11]:
len(df_ipcc_group.doi.drop_duplicates())==len(df_ipcc_group.doi)

True

In [12]:
df_ipbes=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes_visualization.jsonl', lines=True)

In [13]:
df_ipbes

,doi,title,date,creators,tags,countries,concepts,sdg,topics,is_OA_available,year_OA,authors_name,rors,institutions_names,locations_names,locations_id,year
0,10.1890/1051-0761(2000)010[1251:roteka]2.0.co;2,REDISCOVERY OF TRADITIONAL ECOLOGICAL KNOWLEDG...,10/2000,"[{'creatorType': 'author', 'firstName': 'Fikre...","[{'tag': 'ipbes-global_chapter1'}, {'tag': 'ip...","[SE, CA]","[Traditional knowledge, Environmental resource...","[{'id': '15', 'name': 'Life on land'}]","[Conservation, Biodiversity, and Resource Mana...",True,2000,"[[Fikret Berkes, [CA]], [Johan Colding, [SE]],...","[[https://ror.org/02gfys938, CA], [https://ror...","[[University of Manitoba, CA], [Royal Swedish ...",Ecological Applications,1051-0761,2000
1,10.1890/1051-0761(2000)010[1251:roteka]2.0.co;2,REDISCOVERY OF TRADITIONAL ECOLOGICAL KNOWLEDG...,10/2000,"[{'creatorType': 'author', 'firstName': 'Fikre...","[{'tag': 'ipbes-global_chapter1'}, {'tag': 'ip...","[SE, CA]","[Traditional knowledge, Environmental resource...","[{'id': '15', 'name': 'Life on land'}]","[Conservation, Biodiversity, and Resource Mana...",True,2000,"[[Fikret Berkes, [CA]], [Johan Colding, [SE]],...","[[https://ror.org/02gfys938, CA], [https://ror...","[[University of Manitoba, CA], [Royal Swedish ...",Ecological Applications,1051-0761,2000
2,10.1890/1051-0761(2000)010[1251:roteka]2.0.co;2,REDISCOVERY OF TRADITIONAL ECOLOGICAL KNOWLEDG...,10/2000,"[{'creatorType': 'author', 'firstName': 'Fikre...","[{'tag': 'ipbes-global_chapter1'}, {'tag': 'ip...","[SE, CA]","[Traditional knowledge, Environmental resource...","[{'id': '15', 'name': 'Life on land'}]","[Conservation, Biodiversity, and Resource Mana...",True,2000,"[[Fikret Berkes, [CA]], [Johan Colding, [SE]],...","[[https://ror.org/02gfys938, CA], [https://ror...","[[University of Manitoba, CA], [Royal Swedish ...",Ecological Applications,1051-0761,2000
3,10.1016/j.gloenvcha.2013.12.012,Evaluating knowledge exchange in interdiscipli...,2014,"[{'creatorType': 'author', 'firstName': 'Ioan'...",[{'tag': 'ipbes-global_chapter1'}],[GB],"[Knowledge management, Stakeholder, Knowledge ...","[{'id': '15', 'name': 'Life on land'}]","[Sustainability and Climate Change Governance,...",True,2014,"[[Ioan Fazey, [GB]], [Lukas Bunse, [GB]], [Jos...","[[https://ror.org/03h2bxq36, GB], [https://ror...","[[University of Dundee, GB], [University of Du...",Global Environmental Change,0959-3780,2014
4,10.5751/es-07868-200344,Multiple telecouplings and their complex inter...,2015,"[{'creatorType': 'author', 'firstName': 'Jiang...","[{'tag': 'ipbes-global_chapter1'}, {'tag': 'ip...","[CN, AT, US]","[Zhàng, Ecology, Humanities, Philosophy, Mathe...",None,"[Land Use and Ecosystem Services, Species Dist...",True,2015,"[[Jianguo Liu, [US]], [Vanessa Hull, [US]], [J...","[[https://ror.org/05hs6h993, US], [https://ror...","[[Michigan State University, US], [Michigan St...",Ecology and Society,1708-3087,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6358,10.1016/j.tree.2017.06.014,Unifying Research on Social–Ecological Resilie...,2017,"[{'creatorType': 'author', 'firstName': 'Graem...",[{'tag': 'ipbes-global_glossary'}],"[SE, AU]","[Resilience (materials science), Ecological sy...","[{'id': '15', 'name': 'Life on land'}]","[Ecosystem dynamics and resilience, Climate Ch...",True,2017,"[[Graeme S. Cumming, [AU]], [Garry Peterson, [...","[[https://ror.org/028cdc266, AU], [https://ror...",[[ARC Centre of Excellence for Coral Reef Stud...,Trends in Ecology & Evolution,0169-5347,2017
6359,10.2307/2388682,The Structure and Diversity of Rain Forests at...,1991,"[{'creatorType': 'author', 'firstName': 'Don',...",[{'tag': 'ipbes-global_glossary'}],[None],"[Rainforest, Geography, Tropical rain forest, ...","[{'id': '15', 'name': 'Life on land'}]","[Plant and soil sciences, Environmental and Ec...",True,1991,"[[Don Faber‐Langendoen, []], [Alwyn H. Gentry,...",[],[],Biotropica,0006-3606,1991
6360,10.2307/3060213,The Length of the Fallow Period in Tropi

In [14]:
df_ipbes['ipbes']=df_ipbes.apply(lambda row: [{'chapter': str(x['tag']).replace('ipbes-global_chapter','')} for x in row['tags']], axis=1)

In [15]:
df_ipbes.loc[:,'rors']=df_ipbes.loc[:,'rors'].apply(lambda x: [[]] if x==[None] else x)
df_ipbes.loc[:,'rors']=df_ipbes.loc[:,'rors'].apply(lambda x: None if aplatir(x)==[] else x)

In [16]:
df_ipbes_group=df_ipbes.groupby(by=['doi'], dropna=True).agg({'year': lambda x: list(x)[0], 'title': lambda x: list(x)[0], 'rors': lambda x: list(x)[0],'authors_name': lambda x: list(x)[0], 'institutions_names': lambda x: list(x)[0],'ipbes': lambda x: list(x)[0], 'countries': lambda x: list(x)[0], 'topics': lambda x: list(x)[0]})
df_ipbes_group=df_ipbes_group.reset_index()

In [17]:
df_ipbes_group.loc[:,'countries']=df_ipbes_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [18]:
len(df_ipbes_group.doi.drop_duplicates())==len(df_ipbes_group.doi)

True

In [19]:
df_ipcc_ipbes=pd.merge(df_ipcc_group,df_ipbes_group, on='doi', how='outer')

In [20]:
df_ipcc_ipbes['countries']=df_ipcc_ipbes.apply(lambda row: get_xy('countries',row), axis=1)
df_ipcc_ipbes['year']=df_ipcc_ipbes.apply(lambda row: get_xy('year',row), axis=1)
df_ipcc_ipbes['title']=df_ipcc_ipbes.apply(lambda row: get_xy('title',row), axis=1)
df_ipcc_ipbes['rors']=df_ipcc_ipbes.apply(lambda row: get_xy('rors',row), axis=1)
df_ipcc_ipbes['authors_name']=df_ipcc_ipbes.apply(lambda row: get_xy('authors_name',row), axis=1)
df_ipcc_ipbes['institutions_names']=df_ipcc_ipbes.apply(lambda row: get_xy('institutions_names',row), axis=1)
df_ipcc_ipbes['topics']=df_ipcc_ipbes.apply(lambda row: get_xy('topics',row), axis=1)

In [21]:
df_ipcc_ipbes=df_ipcc_ipbes[['doi', 'year','title', 'rors', 'authors_name', 'institutions_names',
       'ipcc', 'countries', 'ipbes','topics']]

In [22]:
len(df_ipcc_ipbes.doi.drop_duplicates())==len(df_ipcc_ipbes.doi)

True

In [23]:
#update_bso_publications()

In [24]:
bso=pd.DataFrame(get_bso_publications())

In [25]:
df_bso=bso.transpose().reset_index()

In [26]:
df_bso.columns=['doi','rors', 'bso_local_affiliations']

In [27]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes,df_bso,on='doi',how='left').rename(columns={'rors_x':'rors'})

In [28]:
del df_ipcc_ipbes_bso['rors_y']

In [29]:
len(df_ipcc_ipbes_bso.doi.drop_duplicates())==len(df_ipcc_ipbes_bso.doi)

True

In [30]:
df_ipcc_ipbes_bso.columns

Index(['doi', 'year', 'title', 'rors', 'authors_name', 'institutions_names',
       'ipcc', 'countries', 'ipbes', 'topics', 'bso_local_affiliations'],
      dtype='object')

In [31]:
df_iso=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\iso2_iso3.json')

In [32]:
#df_iso=df_iso.transpose().reset_index().rename(columns={'index':'code',0:'name'})

In [33]:
dict_iso=df_iso.to_dict(orient='records')[0]
dict_iso['XK']='XKX'

In [34]:
# dict_iso={}
# for i,row in df_iso.iterrows():
#     dict_iso[row['code']]=row['name']
# dict_iso['CW']='Curaçao'
# dict_iso['RS']='Serbia'
# dict_iso['ME']='Montenegro'
# dict_iso['XK']='Kosovo'
# dict_iso['SS']='South Soudan'
# dict_iso['MF']='Saint Martin'

In [35]:
df_ipcc_ipbes_bso.loc[:,'countries']=df_ipcc_ipbes_bso.loc[:,'countries'].apply(lambda x: [dict_iso[y] for y in x] if (x!=None)&(x!=[])&(isinstance(x,float)==False) else None)

In [36]:
#df_brevet_doi=pd.read_json('brevet_doi.json')

In [37]:
#df_ipcc_ipbes_bso_brevet=pd.merge(df_ipcc_ipbes_bso,df_brevet_doi, on='doi', how='left')

In [38]:
#len(df_ipcc_ipbes_bso_brevet.doi.drop_duplicates())==len(df_ipcc_ipbes_bso_brevet.doi)

In [39]:
#df_ipcc_ipbes_bso_brevet.doi.value_counts()[:10]

In [40]:
cached_openalex_data = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data.pkl')
cached_openalex_data_ipbes = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data_ipbes.pkl')
cached_openalex_data.update(cached_openalex_data_ipbes)
df_cache=pd.DataFrame({'doi':list(cached_openalex_data.keys()),'OpenAlex_data':list(cached_openalex_data.values())})

54816 data in cached openalex data
5690 data in cached openalex data


In [41]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes_bso,df_cache, on='doi', how='inner')

In [42]:
len(df_ipcc_ipbes_bso.doi.drop_duplicates())==len(df_ipcc_ipbes_bso.doi)

True

In [43]:
current_dict = df_ipcc_ipbes_bso.to_dict(orient='records')

In [44]:
list_bso_ipcc_ipbes=[]
for i in range(len(df_ipcc_ipbes_bso)):
    if len(current_dict[i]['OpenAlex_data'])==1:
        openalex_info = current_dict[i]['OpenAlex_data'][0]
    else:
        openalex_info = current_dict[i]['OpenAlex_data'][1]
    del current_dict[i]['OpenAlex_data']
    for f in openalex_info:
        current_dict[i]['openalex_'+f] = openalex_info[f]  

In [45]:
df_elastic_input=pd.DataFrame(current_dict)

In [46]:
df_elastic_input=df_ipcc_ipbes_bso

In [47]:
df_elastic_input=df_elastic_input[['doi', 'year', 'title', 'rors', 'ipcc','authors_name', 'institutions_names',
       'countries', 'ipbes','topics']]

In [48]:
df_elastic_input.loc[df_elastic_input.authors_name.apply(lambda x: x==[None]),'authors_name']=None
df_elastic_input.loc[df_elastic_input.countries.apply(lambda x: x==[None]),'countries']=None
df_elastic_input.loc[df_elastic_input.institutions_names.apply(lambda x: x==[None]),'institutions_names']=None
df_elastic_input.loc[df_elastic_input.rors.apply(lambda x: x==[None]),'rors']=None
df_elastic_input.loc[df_elastic_input.authors_name.apply(lambda x: x==[]),'authors_name']=None
df_elastic_input.loc[df_elastic_input.countries.apply(lambda x: x==[]),'countries']=None
df_elastic_input.loc[df_elastic_input.institutions_names.apply(lambda x: x==[]),'institutions_names']=None
df_elastic_input.loc[df_elastic_input.rors.apply(lambda x: x==[]),'rors']=None
df_elastic_input.loc[df_elastic_input.year=='XXXX','year']=None

labs

In [69]:
struct_paysage=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\fr_esr_paysage_structures_all.json')
ror_rnsr_paysage=struct_paysage[['uo_lib','element_ror','identifiant_rnsr']].dropna(subset=['identifiant_rnsr']).rename(columns={'element_ror':'ror'}).dropna(subset=['ror','uo_lib'])

In [70]:
list_ror=ror_rnsr_paysage[['ror','uo_lib']].to_dict(orient='records')

In [71]:
dict_ror={ l['ror']:l['uo_lib'] for l in list_ror}

In [72]:
x=df_elastic_input.rors[100]
[[dict_ror[i[0]],i[1]] if i[0] in list(dict_ror.keys()) else [None,i[1]] for i in x]

[[None, 'CN'],
 ["Laboratoire des sciences du climat et de l'environnement", 'FR'],
 [None, 'FR'],
 [None, 'FR'],
 [None, 'FR'],
 [None, 'CN'],
 ["Laboratoire des sciences du climat et de l'environnement", 'FR'],
 [None, 'FR'],
 [None, 'FR'],
 [None, 'FR'],
 [None, 'US'],
 [None, 'US'],
 [None, 'CN'],
 [None, 'CN'],
 [None, 'CN']]

In [73]:
df_elastic_input['labs']=df_elastic_input.loc[:,'rors'].apply(lambda x: [[dict_ror[i[0]],i[1]] if i[0] in list(dict_ror.keys()) else [None,i[1]] for i in x ]if isinstance(x,list) else None)

C:\Users\haallat\AppData\Local\Temp\ipykernel_19968\701975774.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_elastic_input['labs']=df_elastic_input.loc[:,'rors'].apply(lambda x: [[dict_ror[i[0]],i[1]] if i[0] in list(dict_ror.keys()) else [None,i[1]] for i in x ]if isinstance(x,list) else None)


communes des labos FR

In [74]:
struct_paysage=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\fr_esr_paysage_structures_all.json')
struct_paysage=struct_paysage[struct_paysage.pays_etranger_acheminement=='France']
ror_com_code_paysage=struct_paysage[['element_ror','com_code']].rename(columns={'element_ror':'ror'}).dropna(subset=['ror','com_code'])

In [75]:
list_ror=ror_com_code_paysage[['ror','com_code']].to_dict(orient='records')

In [76]:
dict_ror={ l['ror']:l['com_code'] for l in list_ror}

In [77]:
df_elastic_input['labs_com_code']=df_elastic_input.loc[:,'rors'].apply(lambda x: [[dict_ror[i[0]],i[1]] if i[0] in list(dict_ror.keys()) else [None,i[1]] for i in x ]if isinstance(x,list) else None)

In [78]:
ref_geo=pd.read_json("https://data.enseignementsup-recherche.gouv.fr/api/explore/v2.1/catalog/datasets/fr-esr-referentiel-geographique/exports/json?lang=fr&timezone=Europe%2FBerlin")

In [79]:
ref_geo=ref_geo[['com_code', 'com_id', 'com_nom','uucr_id', 'uucr_nom', 'dep_code', 'dep_id', 'dep_nom', 'aca_code', 'aca_id', 'aca_nom', 'reg_code', 'reg_id', 'reg_nom']]

In [80]:
ref_geo['ref_geo']=ref_geo.apply(lambda row: {'com_code':row['com_code'],'':row['com_id'],'com_nom':row['com_nom'],'uucr_id':row['uucr_id'],'uucr_nom':row['uucr_nom'],'dep_code':row['dep_code'],'dep_id':row['dep_id'],'dep_nom':row['dep_nom'],'aca_code':row['aca_code'],'aca_id':row['aca_id'],'':row['aca_nom'],'':row['aca_nom'],'reg_code':row['reg_code'],'reg_id':row['reg_id'],'reg_nom':row['reg_nom']} ,axis=1)

In [81]:
list_com=ref_geo[['com_code','ref_geo']].to_dict(orient='records')
dict_com={ l['com_code']:l['ref_geo'] for l in list_com}

In [82]:
df_elastic_input['labs_ref_geo']=df_elastic_input.loc[:,'labs_com_code'].apply(lambda x: [[dict_com[i[0]],i[1]] if i[0] in list(dict_com.keys()) else [None,i[1]] for i in x ]if isinstance(x,list) else None)

KeyboardInterrupt: 

In [ ]:
df_elastic_input['french_labs']=df_elastic_input.apply(lambda x: [{"french_lab":x['labs'][i][0], "ref_geo": x['labs_ref_geo'][i][0]} for i in range(len(x['labs'])) if x['labs'][i][1]=='FR' and pd.isna(x['labs'][i][0])==False] if isinstance(x['labs'],list) else None, axis=1)

In [ ]:
len(aplatir(list(df_elastic_input['french_labs'].dropna())))

In [ ]:
df_elastic_input['french_labs'][100]

[{'french_lab': 'Laboratoire de météorologie dynamique',
  'ref_geo': {'com_code': '91477',
   '': 'Versailles',
   'com_nom': 'Palaiseau',
   'uucr_id': 'UU00851',
   'uucr_nom': 'Paris',
   'dep_code': '91',
   'dep_id': 'D091',
   'dep_nom': 'Essonne',
   'aca_code': 25,
   'aca_id': 'A25',
   'reg_code': 11,
   'reg_id': 'R11',
   'reg_nom': 'Île-de-France'}},
 {'french_lab': 'Laboratoire de météorologie dynamique',
  'ref_geo': {'com_code': '91477',
   '': 'Versailles',
   'com_nom': 'Palaiseau',
   'uucr_id': 'UU00851',
   'uucr_nom': 'Paris',
   'dep_code': '91',
   'dep_id': 'D091',
   'dep_nom': 'Essonne',
   'aca_code': 25,
   'aca_id': 'A25',
   'reg_code': 11,
   'reg_id': 'R11',
   'reg_nom': 'Île-de-France'}}]

Communes des labos ou institutions internationals

In [ ]:
data_ror=requests.get(f"https://api.ror.org/organizations/{'018mejw64'}").json()
city=data_ror['addresses'][0]['city']
print(city)
geo_url = f"https://geo.api.gouv.fr/communes?nom={city}&boost=population&limit=1"
geo_response = requests.get(geo_url).json()
geo_response[0]

In [ ]:
def com_labs_and_institutions(ror):
    data_ror=requests.get(f"https://api.ror.org/organizations/{ror}").json()
    city=data_ror['addresses'][0]['city']
    print(city)
    geo_url = f"https://geo.api.gouv.fr/communes?nom={city}&boost=population&limit=1"
    geo_response = requests.get(geo_url).json()
    if len(geo_response)>0:
        return {"com_code": geo_response[0]['city']}
    
#si l'Organization types c'est "Facility" alors c'est labo, sinon c'est une intitutions (regarder si il a des parents)
def labs_or_institutions(ror):
    url=f"https://api.ror.org/organizations/{ror}"
    ror_info=requests.get(url).json()
    if 'Facility' in ror_info['types']:
        return 'lab'
    else:
        [relationship['type'] for relationship in ror_info['relationships'] if relationship['type']=='parent']

institutions

In [67]:
ror_rnsr_paysage.columns

Index(['uo_lib', 'ror', 'identifiant_rnsr'], dtype='object')

In [68]:
list_ror=ror_rnsr_paysage[['ror','identifiant_rnsr']].to_dict(orient='records')
dict_ror={ l['ror']:l['identifiant_rnsr'] for l in list_ror}

In [69]:
rnsr_tutelles_scanr=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\organizations.jsonl', lines=True)
rnsr_tutelles_scanr['tutelles_label']=rnsr_tutelles_scanr['institutions'].apply(lambda x: [y.get('label') for y in x] if len(x)>0 else None)
rnsr_tutelles_scanr['tutelles_id']=rnsr_tutelles_scanr['institutions'].apply(lambda x: [y.get('structure') for y in x] if len(x)>0 else None)
rnsr_tutelles_scanr.loc[:,'ror']=rnsr_tutelles_scanr.loc[:,'externalIds'].apply(lambda y: [x.get('id') for x in y if x.get('type')=='ROR'])
rnsr_tutelles_scanr.loc[:,'rnsr']=rnsr_tutelles_scanr.loc[:,'externalIds'].apply(lambda y: [x.get('id') for x in y if x.get('type')=='rnsr'])
rnsr_tutelles=rnsr_tutelles_scanr[['id','tutelles_label','tutelles_id']].rename(columns={'id':'rnsr'}).dropna()
ror_tutelles=rnsr_tutelles_scanr[['ror','tutelles_label','tutelles_id']].explode('ror').dropna()
rnsr_tutelles_siren=rnsr_tutelles_scanr[['id','tutelles_id']].rename(columns={'id':'rnsr'}).dropna()
ror_tutelles_siren=rnsr_tutelles_scanr[['ror','tutelles_id']].explode('ror').dropna()

In [70]:
list_tutelles_rnsr=rnsr_tutelles.to_dict(orient='records')
dict_tutelles_rnsr={l['rnsr']:l['tutelles_label'] for l in list_tutelles_rnsr}
list_tutelles_ror=ror_tutelles.to_dict(orient='records')
dict_tutelles_ror={l['ror']:l['tutelles_label'] for l in list_tutelles_ror}

In [71]:
list_tutelles_id_rnsr=rnsr_tutelles_siren.to_dict(orient='records')
dict_tutelles_id_rnsr={l['rnsr']:l['tutelles_id'] for l in list_tutelles_id_rnsr}
list_tutelles_id_ror=ror_tutelles.to_dict(orient='records')
dict_tutelles_id_ror={l['ror']:l['tutelles_id'] for l in list_tutelles_id_ror}

In [72]:
df_elastic_input['rnsr']=df_elastic_input.loc[:,'rors'].apply(lambda x: [[dict_ror[i[0]],i[1]] if i[0] in list(dict_ror.keys()) else [None,i[1]] for i in x ]if isinstance(x,list) else None)

In [73]:
df_elastic_input['tutelles_label_rnsr']=df_elastic_input.loc[:,'rnsr'].apply(lambda x: [[dict_tutelles_rnsr[i[0]],i[1]] if i[0] in list(dict_tutelles_rnsr.keys()) else [None,i[1]] for i in x ]if isinstance(x,list) else None)
df_elastic_input['tutelles_label_ror']=df_elastic_input.loc[:,'rors'].apply(lambda x: [[dict_tutelles_ror[i[0]],i[1]] if i[0] in list(dict_tutelles_ror.keys()) else [None,i[1]] for i in x ]if isinstance(x,list) else None)

In [74]:
df_elastic_input['tutelles_siren_rnsr']=df_elastic_input.loc[:,'rnsr'].apply(lambda x: [[dict_tutelles_id_rnsr[i[0]],i[1]] if i[0] in list(dict_tutelles_rnsr.keys()) else [None,i[1]] for i in x ]if isinstance(x,list) else None)
df_elastic_input['tutelles_siren_ror']=df_elastic_input.loc[:,'rors'].apply(lambda x: [[dict_tutelles_id_ror[i[0]],i[1]] if i[0] in list(dict_tutelles_ror.keys()) else [None,i[1]] for i in x ]if isinstance(x,list) else None)

In [75]:
def get_tutelle(row, label='label'):
    tutelles=[]
    if isinstance(row[f"tutelles_{label}_rnsr"],list):
        for i in range(len(row[f"tutelles_{label}_rnsr"])):
            if isinstance(row[f"tutelles_{label}_rnsr"][i][0],list):
                tutelles.append([row[f"tutelles_{label}_rnsr"][i][0],row[f"tutelles_{label}_rnsr"][i][1]])
            elif isinstance(row[f"tutelles_{label}_ror"][i][0],list):
                tutelles.append([row[f"tutelles_{label}_ror"][i][0],row[f"tutelles_{label}_ror"][i][1]])
            else:
                tutelles.append([None,row[f"tutelles_{label}_ror"][i][1]])
        return tutelles
    else:
        return None

In [81]:
df_elastic_input['tutelles_name']=df_elastic_input.apply( lambda row: get_tutelle(row) ,axis=1)

In [77]:
df_elastic_input['tutelles_siren']=df_elastic_input.apply( lambda row: get_tutelle(row, label='siren') ,axis=1)

In [79]:
df_elastic_input.tutelles_siren[100]

[[None, 'CN'],
 [None, 'CN'],
 [None, 'DE'],
 [None, 'CN'],
 [None, 'CN'],
 [None, 'CN'],
 [None, 'CN'],
 [None, 'NO'],
 [None, 'NO'],
 [None, 'NO'],
 [['197534597',
   '197535016',
   '199115684',
   '180089013',
   '130023385',
   '130025620',
   '130026149'],
  'FR'],
 [None, 'FR'],
 [None, 'FR'],
 [None, 'FR'],
 [None, 'FR'],
 [['197534597',
   '197535016',
   '199115684',
   '180089013',
   '130023385',
   '130025620',
   '130026149'],
  'FR'],
 [None, 'US'],
 [None, 'US'],
 [None, 'DE'],
 [None, 'NO'],
 [None, 'US'],
 [None, 'JP'],
 [None, 'DK']]

In [ ]:
df_elastic_input['french_tutelles']=df_elastic_input.loc[:,'tutelles'].apply(lambda x: aplatir(list(pd.Series([i[0] for i in x if i[1]=='FR' and isinstance(i[0],list)]))) if isinstance(x,list) else None)

In [ ]:
df_elastic_input['french_tutelles'][100]

In [ ]:
pd.Series(aplatir(list(df_elastic_input['french_tutelles'].dropna()))).value_counts()

communes institutions

In [80]:
from dotenv import load_dotenv
from code_utils.utils import get_headers  
load_dotenv()

def get_code_commune_siege(siren):
    try:
        headers = get_headers(base="https://api.insee.fr/entreprises/sirene/V3.11/")
        url = f"https://api.insee.fr/entreprises/sirene/V3.11/siren/{siren}"
        response = requests.get(url, headers=headers)
        data=response.json()
        unite_legale = data.get("uniteLegale", {})
        periode_actuelle = unite_legale.get("periodesUniteLegale", [])[0]
        nic_siege = periode_actuelle.get("nicSiegeUniteLegale")
        siret_siege = siren + nic_siege
        url =f"https://api.insee.fr/entreprises/sirene/V3.11/siret/{siret_siege}"
        response = requests.get(url, headers=headers)
        data=response.json()
        return data["etablissement"]["adresseEtablissement"]["codeCommuneEtablissement"]
    except:
        return None 

In [ ]:
df_elastic_input['tutelles']=df_elastic_input.apply( lambda row: {'name': str(get_tutelle(row)), 'siren': str(get_tutelle(row, label='siren')), 'code_com': get_code_commune_siege(get_tutelle(row, label='siren'))} ,axis=1)

In [ ]:
df_elastic_input['tutelles'][100]

In [ ]:
list_org_nat=['Centre national de la recherche scientifique', 'Institut de recherche pour le developpement',"Commissariat à l'énergie atomique et aux énergies alternatives","Institut national de recherche pour l'agriculture, l'alimentation et l'environnement"]

In [ ]:
def com_institutions(row):
    liste=[]
    if isinstance(row['tutelles_com_code'],list):
        for i in range(len(row['tutelles_com_code'])):
            if row['tutelles_com_code'][i][0] in list(dict_com.keys()) and row['tutelles'][i][0] not in list_org_nat:
                liste.append([dict_com[row['tutelles_com_code'][i][0]],row['tutelles_com_code'][i][1]]) 
            elif row['tutelles_com_code'][i][0] in list(dict_com.keys()) and row['tutelles'][i][0] in list_org_nat:
                liste.append(['Organisme national',row['tutelles_com_code'][i][1]])
            else:
                liste.append([None,row['tutelles_com_code'][i][1]])
        return liste
    else :
        return None

In [ ]:
df_elastic_input['tutelles_ref_geo']=df_elastic_input.apply(lambda row: com_institutions(row), axis=1)

into json to push into the cluster

In [ ]:
aplatir(list(df_elastic_input[pd.isna(df_elastic_input.countries)].french_labs.dropna()))

In [ ]:
aplatir(list(df_elastic_input[pd.isna(df_elastic_input.countries)].french_tutelles.dropna()))

In [ ]:
df_elastic_input=df_elastic_input.dropna(subset=['countries'])

In [ ]:
df_elastic_input[df_elastic_input.countries.apply(lambda x: 'FRA' in x)].reset_index()#.doi.to_json('list_doi_fr.json',orient='records')

In [ ]:
df_elastic_input.french_tutelles[37]

In [ ]:
df_elastic_input.rors[100]

In [ ]:
dict_doi_bso=[]
for i,row in df_elastic_input.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    #dict_row3=remove_too_long(dict_row2)
    dict_doi_bso.append(dict_row2)

In [ ]:
dict_doi_bso[0].keys()

In [ ]:
len(dict_doi_bso)

In [ ]:
json_lines='\n'.join(json.dumps(entry) for entry in dict_doi_bso)
with open('bso_ipcc_ipbes_elastic.jsonl', 'w') as f:
      f.write(json_lines)

fichier excel

In [ ]:
df=pd.read_json('bso_ipcc_ipbes_elastic.jsonl', lines=True)

In [ ]:
df=df[['doi','ipcc','ipbes']]

In [ ]:
df.loc[:,'ipbes']=df.loc[:,'ipbes'].apply(lambda x: [y['chapter'] for y in x] if isinstance(x, list) else [])

In [ ]:
df.loc[:,'ipcc']=df.loc[:,'ipcc'].apply(lambda x: [y['name'] for y in x] if isinstance(x, list) else [])

In [ ]:
df.loc[:,'doi']=df.loc[:,'doi'].apply(lambda x: f"doi{x}")

In [ ]:
df.drop_duplicates('doi')

In [ ]:
with open('ipcc_ipbes_chapters.jsonl', 'w') as f:
    for record in df.to_dict(orient='records'):
        record2 = {k: v for k, v in record.items() if isinstance(v, list) or isinstance(v, str)}
        json.dump(record2, f)
        f.write('\n')

In [ ]:
df.loc[:,'rors']=df.loc[:,'rors'].apply(lambda x: [[]] if x==[None] else x)
df.loc[:,'authors_name']=df.loc[:,'authors_name'].apply(lambda x: [[]] if x==[None] else x)
df.loc[:,'institutions_names']=df.loc[:,'institutions_names'].apply(lambda x: [[]] if x==[None] else x)

In [ ]:
df.loc[pd.isna(df['rors'])==False,'rors']=df.loc[pd.isna(df['rors'])==False,'rors'].apply(lambda x: ','.join([j.replace('https://ror.org/','') for j in list(pd.Series(aplatir([list(y[i].keys()) for y in x for i in range(len(y))])).drop_duplicates())]))

In [ ]:
df.loc[pd.isna(df['institutions_names'])==False,'institutions_names']=df.loc[pd.isna(df['institutions_names'])==False,'institutions_names'].apply(lambda x: ','.join(list(pd.Series(aplatir([list(y[i].keys()) for y in x for i in range(len(y))])).drop_duplicates())))

In [ ]:
df.loc[pd.isna(df['authors_name'])==False,'authors_name']=df.loc[pd.isna(df['authors_name'])==False,'authors_name'].apply(lambda x: ','.join(list(pd.Series(aplatir([list(y.keys()) for y in x if isinstance(y,dict)])).drop_duplicates())))

In [ ]:
df.loc[pd.isna(df['ipcc'])==False,'ipcc']=df.loc[pd.isna(df['ipcc'])==False,'ipcc'].apply(lambda x: ','.join(list((pd.Series([y.get('name') for y in x]).drop_duplicates()))))

In [ ]:
df.loc[pd.isna(df['ipbes'])==False,'ipbes']=df.loc[pd.isna(df['ipbes'])==False,'ipbes'].apply(lambda x: ','.join(list((pd.Series([y.get('chapter') for y in x]).drop_duplicates()))))

In [ ]:
df['is_fr']=df['countries'].apply(lambda x: 'France' in x if isinstance(x,list) else False)

In [ ]:
df.loc[pd.isna(df['countries'])==False,'countries']=df.loc[pd.isna(df['countries'])==False,'countries'].apply(lambda x: ','.join(x))

In [ ]:
df[['doi','title','ipcc','ipbes','countries','is_fr','rors','institutions_names','authors_name']].to_excel('doi_ipcc_ipbes.xlsx', index=False)